# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [14]:
# import libraries
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import csv
import os
from sqlalchemy import create_engine
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
import pickle
from sklearn.externals import joblib

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
df = pd.read_csv('cleanedData.csv',sep=',',encoding='utf-8')
df.head()
X = df['message'].astype(str)
Y = df.drop(columns=['message', 'original', 'id','genre']).astype(str)
print(X.head())
print(Y.head())

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object
    genre related request offer aid_related medical_help medical_products  \
0  direct       1       0     0           0            0                0   
1  direct       1       0     0           1            0                0   
2  direct       1       0     0           0            0                0   
3  direct       1       1     0           1            0                1   
4  direct       1       0     0           0            0                0   

  search_and_rescue security military      ...      aid_centers  \
0                 0        0        0      ...                0   
1                 0        0        0      ...                0   
2                 0        0    

In [9]:
# load data from database
database_filepath = "cleanedDataBase1"
engine = create_engine(f'sqlite:///{os.path.abspath(database_filepath)}')
df = pd.read_sql_table('CleanedData',con=engine)  
X = df['message'].astype(str)
Y = df.drop(columns=['message', 'original', 'id', 'genre']).astype(str)

In [10]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [11]:
Y.head()
print(Y['related'].unique())

['1' '0']


### 2. Write a tokenization function to process your text data

In [12]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [19]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=tokenize ,max_df=0.95, min_df=5)),  # Convert text to numerical features
    ('clf', MultiOutputClassifier(RandomForestClassifier()))  # Multi-output classification
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [24]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.95, max_features=None, min_df=5,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [21]:
X_train.head()

15272    The failure of rescuers to reach these areas p...
15797    NEW YORK, 21 September 2007 - The exceptionall...
7072     Human rights group Be a pillar on which those ...
3261          Please send us information concerning them. 
23655    In deference to the concerns of the Malian gov...
Name: message, dtype: object

In [22]:
Y_train.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
15272,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15797,1,0,0,1,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,0
7072,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3261,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23655,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [17]:
model = joblib.load("best_model.pkl")

In [29]:
Y_pred = pipeline.predict(X_test)

for i, col in enumerate(Y_test.columns):
    print(f"Category: {col}")
    score = classification_report(Y_test.iloc[:, i], Y_pred[:, i])
    print(score)

Category: related
             precision    recall  f1-score   support

          0       0.60      0.34      0.44      1247
          1       0.82      0.93      0.87      3959

avg / total       0.77      0.79      0.77      5206

Category: request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      4304
          1       0.83      0.42      0.55       902

avg / total       0.88      0.88      0.87      5206

Category: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5181
          1       0.00      0.00      0.00        25

avg / total       0.99      1.00      0.99      5206

Category: aid_related
             precision    recall  f1-score   support

          0       0.74      0.86      0.80      3050
          1       0.75      0.58      0.65      2156

avg / total       0.75      0.75      0.74      5206

Category: medical_help
             precision    recall  f1-score 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.98      1.00      0.99      5100
          1       0.00      0.00      0.00       106

avg / total       0.96      0.98      0.97      5206

Category: military
             precision    recall  f1-score   support

          0       0.97      1.00      0.98      5031
          1       0.59      0.07      0.13       175

avg / total       0.96      0.97      0.95      5206

Category: child_alone
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5206

avg / total       1.00      1.00      1.00      5206

Category: water
             precision    recall  f1-score   support

          0       0.96      1.00      0.98      4862
          1       0.86      0.39      0.53       344

avg / total       0.95      0.96      0.95      5206

Category: food
             precision    recall  f1-score   support

          0       0.93      0.99      0.96      4620
          1   

### 6. Improve your model
Use grid search to find better parameters. 

In [30]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
parameters = {
    'clf__estimator__n_estimators': [50, 100],  # Number of trees in RandomForest
    'clf__estimator__max_depth': [10, None]  # Maximum depth of trees
}

grid_search = GridSearchCV(pipeline, parameters, cv=2, verbose=3, n_jobs=-1)

grid_search.fit(X_train, Y_train)

optimzed_model = grid_search.best_estimator_

# Step 8: Display the best parameters
print("Best Parameters:", grid_search.best_params_)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=50 ...
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=50 ...
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=100 ..
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=100 ..
[CV] clf__estimator__max_depth=None, clf__estimator__n_estimators=50 .
[CV] clf__estimator__max_depth=None, clf__estimator__n_estimators=50 .
[CV] clf__estimator__max_depth=None, clf__estimator__n_estimators=100 
[CV] clf__estimator__max_depth=None, clf__estimator__n_estimators=100 
[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=50, score=0.18759004898664874, total= 2.8min
[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=50, score=0.19613869945250217, total= 3.0min
[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=100, score=0.19604264720007683, total= 4.3min
[CV]  clf__estimator__max_depth=10, clf__e

[Parallel(n_jobs=-1)]: Done   4 out of   8 | elapsed:  5.2min remaining:  5.2min


[CV]  clf__estimator__max_depth=None, clf__estimator__n_estimators=50, score=0.240899049082701, total= 7.8min
[CV]  clf__estimator__max_depth=None, clf__estimator__n_estimators=50, score=0.24906349053885313, total= 7.9min
[CV]  clf__estimator__max_depth=None, clf__estimator__n_estimators=100, score=0.24320430314090866, total=10.3min
[CV]  clf__estimator__max_depth=None, clf__estimator__n_estimators=100, score=0.2561713572183268, total=10.3min


[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed: 11.0min finished


Best Parameters: {'clf__estimator__max_depth': None, 'clf__estimator__n_estimators': 100}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [31]:
#Y_pred = grid_search.best_estimator_.predict(X_test)
Y_pred = optimzed_model.predict(X_test)
# Step 10: Evaluate the model
for i, col in enumerate(Y_test.columns):
    print(f"Category: {col}")
    print(classification_report(Y_test.iloc[:, i], Y_pred[:, i]))

Category: related
             precision    recall  f1-score   support

          0       0.76      0.27      0.40      1247
          1       0.81      0.97      0.88      3959

avg / total       0.80      0.81      0.77      5206

Category: request
             precision    recall  f1-score   support

          0       0.90      0.99      0.94      4304
          1       0.88      0.48      0.62       902

avg / total       0.90      0.90      0.89      5206

Category: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5181
          1       0.00      0.00      0.00        25

avg / total       0.99      1.00      0.99      5206

Category: aid_related
             precision    recall  f1-score   support

          0       0.78      0.86      0.81      3050
          1       0.76      0.65      0.70      2156

avg / total       0.77      0.77      0.77      5206

Category: medical_help
             precision    recall  f1-score 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.98      1.00      0.99      5100
          1       0.00      0.00      0.00       106

avg / total       0.96      0.98      0.97      5206

Category: military
             precision    recall  f1-score   support

          0       0.97      1.00      0.98      5031
          1       0.88      0.09      0.16       175

avg / total       0.97      0.97      0.96      5206

Category: child_alone
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5206

avg / total       1.00      1.00      1.00      5206

Category: water
             precision    recall  f1-score   support

          0       0.96      1.00      0.98      4862
          1       0.92      0.40      0.56       344

avg / total       0.96      0.96      0.95      5206

Category: food
             precision    recall  f1-score   support

          0       0.95      0.98      0.97      4620
          1   

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [32]:
print(X_test.head(2))
print(Y_test.head(2))
Y_pred = pipeline.predict(X_test.head(2))
print(Y_pred)

20861    Light showers may bring additional relief duri...
12293    This place is pretty empty considering the sto...
Name: message, dtype: object
      related request offer aid_related medical_help medical_products  \
20861       0       0     0           0            0                0   
12293       1       0     0           1            0                0   

      search_and_rescue security military child_alone      ...       \
20861                 0        0        0           0      ...        
12293                 0        0        0           0      ...        

      aid_centers other_infrastructure weather_related floods storm fire  \
20861           0                    0               0      0     0    0   
12293           0                    0               1      0     1    0   

      earthquake cold other_weather direct_report  
20861          0    0             0             0  
12293          0    0             0             0  

[2 rows x 36 columns]
[['0' '0' '

### 9. Export your model as a pickle file

In [33]:
with open('best_model.pkl', 'wb') as f:
    pickle.dump(grid_search.best_estimator_, f)

print("Model saved as best_model.pkl")

Model saved as best_model.pkl


In [16]:
# Load with pickle
with open('best_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.